In [15]:
import overpy
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
city_name = 'rouen'

In [4]:
city_boundary_fp = f"./{city_name}_project/{city_name}_city_boundary.gpkg"

### 1st step: Create circular buffer around city boundary
- Calculate centroid
- Reproject to utm
- Measure max distance to city boundary polygon
- Calculare 125% the distance of max distance
- Buffer centroid
- Reproject to Geopgraphic and save **<name_of_city>_city_buffer.gpkg** file.

In [56]:
city_boundary = gpd.read_file(city_boundary_fp)

In [57]:
city_boundary = city_boundary.dissolve()

In [58]:
city_boundary.explore()

In [59]:
centroid = city_boundary.centroid[0]

/var/folders/r8/h0z48l2x5z9348h1bsn12gfr0000gq/T/ipykernel_34699/313260075.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = city_boundary.centroid[0]


In [60]:
city_boundary_points_geom = [Point(x) for x in city_boundary['geometry'].explode(index_parts=True)[0][0].exterior.coords]

In [61]:
max_distance = max([x.distance(centroid) for x in city_boundary_points_geom])

In [62]:
buffer_25_geom = centroid.buffer(max_distance*0.25 + max_distance)
# buffer_30_geom = centroid.buffer(max_distance*0.30 + max_distance)

In [63]:
buffer_geom = centroid.buffer(max_distance)
buffer = gpd.GeoDataFrame(geometry=[buffer_30_geom])
buffer = buffer.set_crs(4326)

In [64]:
buffer.to_file(f"./{city_name}_project/{city_name}_city_buffer.gpkg", driver="GPKG")

### 2nd STEP: Create Outside Polygon

In [283]:
city_boundary['geometry'] = city_boundary['geometry'].make_valid()

In [284]:
outside_boundary = buffer.symmetric_difference(city_boundary['geometry'][0])[0]

### 3rd STEP: Retrieve building footprints from OSM using Overpass API
- Retrieve building footprints
- Dissolve
- Export file

In [146]:
bf = ox.geometries.geometries_from_polygon(city_boundary['geometry'][0], tags = {'building': True})[['geometry']]

ReadTimeout: HTTPSConnectionPool(host='overpass-api.de', port=443): Read timed out. (read timeout=180)

In [ ]:
bf = bf.loc[(bf.geometry.geom_type=='Polygon') | (bf.geometry.geom_type=='MultiPolygon')]

In [ ]:
bf = bf.dissolve()

In [ ]:
bf.explore()

In [ ]:
bf.to_file(f"./{city_name}_project/{city_name}_city_bf.gpkg", driver="GPKG")

### 4th STEP:  City streets and Outside Streets

In [16]:
streets = ox.geometries.geometries_from_polygon(buffer['geometry'][0], tags = {'highway': True})[['geometry']]

In [65]:
streets = ox.geometries.geometries_from_polygon(buffer['geometry'][0], tags = {'highway': True})

In [66]:
highway_key_values = ['bridleway', 'cycleway', 'living_street', 'motorwat', 'motorway_link',
                      'pedestrian', 'primary', 'primary_link', 'residential', 'secondary', 'secondary_link',
                      'tertiary', 'tertiary_link', 'trunk']

In [67]:
streets_filtered = streets[streets['highway'].isin(highway_key_values)]

In [68]:
streets_filtered.head(2)

highway  \
element_type osmid              
way          4042795  primary   
             4042798  primary   

                                                               geometry  \
element_type osmid                                                        
way          4042795  LINESTRING (1.01416 49.33949, 1.01423 49.33946...   
             4042798  LINESTRING (1.07217 49.43470, 1.07226 49.43471...   

                     enforcement     ref created_by crossing crossing_ref  \
element_type osmid                                                          
way          4042795         NaN     NaN        NaN      NaN          NaN   
             4042798         NaN  N 2338        NaN      NaN          NaN   

                     traffic_sign source tactile_paving  ...  \
element_type osmid                                       ...   
way          4042795          NaN    NaN            NaN  ...   
             4042798          NaN    NaN            NaN  ...   

                     parking:lane:right:parallel parking:lane:forward  \
element_type osmid                                                      
way          4042795                         NaN                  NaN   
             4042798                         NaN                  NaN   

                     parking:lane:right:perpendicular lanes:bus:backward  \
element_type osmid                                                         
way          4042795                              NaN                NaN   
             4042798                              NaN                NaN   

                     construction_end_expected structure bicycle_road private  \
element_type osmid                                                              
way          4042795                       NaN       NaN          NaN     NaN   
             4042798                       NaN       NaN          NaN     NaN   

                     ways type  
element_type osmid              
way          4042795  NaN  NaN  
             4042798  NaN  NaN  

[2 rows x 330 columns]

In [69]:
streets_filtered['geometry'].type.value_counts()

LineString    11687
Polygon         230
dtype: int64

In [20]:
streets['highway'].unique()

array(['motorway_junction', 'traffic_signals', 'emergency_bay',
       'crossing', 'mini_roundabout', 'stop', 'give_way',
       'turning_circle', 'street_lamp', 'bus_stop', 'turning_loop',
       'footway', 'elevator', 'steps', 'speed_camera', 'passing_place',
       'traffic_mirror', 'milestone', 'primary', 'trunk_link', 'motorway',
       'motorway_link', 'trunk', 'secondary', 'unclassified',
       'residential', 'tertiary', 'track', 'service', 'living_street',
       'secondary_link', 'pedestrian', 'primary_link', 'path',
       'construction', 'cycleway', 'tertiary_link', 'platform', 'road',
       'raceway', 'bridleway', 'proposed', 'services', 'busway',
       'corridor'], dtype=object)

In [70]:
# streets_filtered = streets.copy()
streets_filtered = streets_filtered.loc[streets['geometry'].geom_type=='LineString']

In [71]:
streets_filtered['geometry'].type.value_counts()

LineString    11687
dtype: int64

In [72]:
streets_filtered.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [73]:
buffer.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [74]:
streets_filtered_buffer_clip = gpd.clip(streets_filtered, buffer, keep_geom_type=True)
streets_filtered_buffer_clip = streets_filtered_buffer_clip.dissolve()

/Users/rllop/opt/anaconda3/envs/ox/lib/python3.11/site-packages/geopandas/tools/clip.py:67: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  clipped.loc[


In [75]:
streets_filtered_buffer_clip = streets_filtered_buffer_clip[['geometry', 'highway']]
streets_filtered_buffer_clip

,geometry,highway
0,"MULTILINESTRING ((1.02732 49.31420, 1.02803 49...",residential


In [76]:
streets_filtered_buffer_clip.to_file(f"./{city_name}_project/{city_name}_map_streets.gpkg", driver="GPKG")

In [77]:
streets_filtered_buffer_clip.explore()

In [293]:
# city_streets = gpd.clip(streets_filtered, city_boundary, keep_geom_type=True)
# city_streets = city_streets.dissolve()

In [43]:
# city_streets.geom_type.value_counts()

In [295]:
# city_streets.to_file(f"./{city_name}_project/{city_name}_city_streets.gpkg", driver="GPKG")

In [296]:
# outside_streets_filtered = gpd.clip(streets_filtered, outside_boundary, keep_geom_type=True)
# outside_streets_filtered = outside_streets_filtered.dissolve()

In [297]:
# outside_streets_filtered.to_file(f"./{city_name}_project/{city_name}_streets_outside.gpkg", driver="GPKG")

### 5th STEP: Water

In [134]:
water = ox.geometries.geometries_from_polygon(buffer['geometry'][0], tags = {'water': True})

In [135]:
water.explore()

In [136]:
water_exploded=water.explode()

/var/folders/r8/h0z48l2x5z9348h1bsn12gfr0000gq/T/ipykernel_34699/1869629373.py:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  water_exploded=water.explode()


In [137]:
water_exploded['geometry'].type.value_counts()

Polygon    256
dtype: int64

In [138]:
water_exploded['geometry'].type.value_counts()

Polygon    256
dtype: int64

In [139]:
water_exploded = gpd.clip(water_exploded, buffer, keep_geom_type=False)

/Users/rllop/opt/anaconda3/envs/ox/lib/python3.11/site-packages/geopandas/tools/clip.py:67: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  clipped.loc[


In [140]:
water_exploded.columns

Index(['source', 'name', 'intermittent', 'natural', 'nodes', 'source_ref',
       'water', 'salt', 'wall', 'access', 'layer', 'tunnel', 'basin',
       'landuse', 'seamark:type', 'operator', 'ref', 'barrier', 'ways', 'type',
       'boat', 'motorboat', 'tidal', 'harbour', 'geometry'],
      dtype='object')

In [141]:
water_exploded = water_exploded[['geometry', 'source']]

In [142]:
water_exploded.explore()

In [143]:
water_exploded = water_exploded.dissolve()

In [144]:
water_exploded.to_file(f"./{city_name}_project/water.gpkg", driver="GPKG")